# Reinforcement Learning control strategies for Electric Vehicles fleet Virtual Power Plants
Thesis based on the development of a RL agent that manages a VPP through EVs charging stations in an household environment. Main optimization objectives of the VPP are: Valley filling, peak shaving and zero resulting load over time. Main action performed to reach objectives are: storage of Renewable energy resources and power push in the grid at high demand times. The development of the Virtual Power Plant environment is based on the ELVIS (Electric Vehicles Infrastructure Simulator) open library from DAI-Labor: https://github.com/dailab/elvis The thesis code is currently available at: (https://github.com/francescomaldonato/RL_VPP_Thesis)

Author: Francesco Maldonato

## VPP experiment tester Notebook based on EVs arrival, with StableBaselines3 trained model (Recurrent PPO) [10-15-20-25-30-35 EVs per week test]

Installing required packages and dependencies

In [1]:
%%capture
!pip install py-elvis==0.2.1
!pip install pyyaml==5.4
!pip install plotly==5.9.0
!pip install -U kaleido==0.2.1

!pip install stable-baselines3[extra]==1.6.1
!pip install stable-baselines==1.6.1
!pip install sb3-contrib==1.6.1
!pip install gym==0.20.0
!pip install -q wandb==0.13.4

In [3]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/
%ls

In [4]:
import yaml
import numpy as np
import pandas as pd
from VPP_environment import VPPEnv, VPP_Scenario_config
from elvis.config import ScenarioConfig
import os
import torch
import random
import wandb
from sb3_contrib import RecurrentPPO #The available algoritmhs in sb3-contrib for the custom environment with MultiInputPolicy
from sb3_contrib.common.maskable.utils import get_action_masks
import stable_baselines3 as sb3
from stable_baselines3.common.env_checker import check_env

#Check if cuda device is available for training
print("Torch-Cuda available device:", torch.cuda.is_available())
print(sb3.get_system_info())
#!wandb --version

Torch-Cuda available device: False
OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022
Python: 3.7.14
Stable-Baselines3: 1.6.1
PyTorch: 1.12.1+cu113
GPU Enabled: False
Numpy: 1.21.6
Gym: 0.20.0

({'OS': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022', 'Python': '3.7.14', 'Stable-Baselines3': '1.6.1', 'PyTorch': '1.12.1+cu113', 'GPU Enabled': 'False', 'Numpy': '1.21.6', 'Gym': '0.20.0'}, 'OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022\nPython: 3.7.14\nStable-Baselines3: 1.6.1\nPyTorch: 1.12.1+cu113\nGPU Enabled: False\nNumpy: 1.21.6\nGym: 0.20.0\n')


In [5]:
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

## Load ELVIS YAML config file
Section where the EVs arrival simulation parameters are loaded through the Yaml config file from the 'data/config_builder/' folder.

In [6]:
#Loading paths for input data
current_folder = ''
VPP_training_data_input_path = current_folder + 'data/data_training/environment_table/' + 'Environment_data_2019.csv'
VPP_testing_data_input_path = current_folder + 'data/data_testing/environment_table/' + 'Environment_data_2020.csv'
VPP_validating_data_input_path = current_folder + 'data/data_validating/environment_table/' + 'Environment_data_2018.csv'
elvis_input_folder = current_folder + 'data/config_builder/'

#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

print(elvis_config_file)
print(VPP_config_file)

Vehicle types: <generator object ScenarioConfig.__str__.<locals>.<genexpr> at 0x7efb82049f50>Mean parking time: 23.99
Std deviation of parking time: 1
Mean value of the SOC distribution: 0.5
Std deviation of the SOC distribution: 0.1
Max parking time: 24
Number of charging events per week: 35
Vehicles are disconnected only depending on their parking time
Queue length: 0
Opening hours: None
Scheduling policy: Uncontrolled

{'start_date': '2022-01-01T00:00:00', 'end_date': '2023-01-01T00:00:00', 'resolution': '0:15:00', 'num_households': 4, 'solar_power': 16, 'wind_power': 12, 'EV_types': [{'battery': {'capacity': 100, 'efficiency': 1, 'max_charge_power': 150, 'min_charge_power': 0}, 'brand': 'Tesla', 'model': 'Model S', 'probability': 1}], 'charging_stations_n': 4, 'EVs_n': 35, 'EVs_n_max': 1827, 'mean_park': 23.99, 'std_deviation_park': 1, 'EVs_mean_soc': 50.0, 'EVs_std_deviation_soc': 10.0, 'EV_load_max': 44, 'EV_load_rated': 14.8, 'EV_load_min': 1, 'houseRWload_max': 10, 'av_max_ener

In [7]:
#TESTING Environment initialization
env = VPPEnv(VPP_validating_data_input_path, elvis_config_file, VPP_config_file)
#env.plot_VPP_input_data()
#Function to check custom environment and output additional warnings if needed
check_env(env)
#env.plot_reward_functions()

Charging event: 1, Arrival time: 2022-01-01 13:30:00, Parking_time: 24, Leaving_time: 2022-01-02 13:30:00, SOC: 0.4506687067359438, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 1825, Arrival time: 2022-12-31 15:15:00, Parking_time: 23.966819721963585, Leaving_time: 2023-01-01 15:13:00.550999, SOC: 0.4564269518803252, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -30085.39 , over-consume=kWh  2136.67 , under-consume=kWh  -32222.06 , Total_cost=€  -1187.15 , overcost=€  113.34
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  24672.87 , over-consume=kWh  43710.87 , under-consume=kWh  19038.0 , Total_cost=€  1150.77 , overcost=€  1923.14 , Charging_events=  1825 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  67.66
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  26465.99 , over-consume=kWh  45819.29 , under-consume=kWh  19353.3 , Total_cost=€  1217.85

In [8]:
#Loading training model, from local directory or from wandb previous trainings
RecurrentPPO_path = "trained_models/RecurrentPPO_models/model_RecurrentPPO_"

#model_id = "s37o8q0n"
model_id = "333ckz0i"
model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

# run_id_restore = "2y2dqvyn"
# model = wandb.restore(f'model_{run_id_restore}.zip', run_path=f"francesco_maldonato/RL_VPP_Thesis/{run_id_restore}")
results_table = []

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/save_util.py:169: UserWarning: Could not deserialize object _last_lstm_states. Consider using `custom_objects` argument to replace this object.
  + "this object."


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Validating dataset VPP Simulation using the loaded trained model [35 EVs per week]

In [9]:
#TEST Model
episodes = 10
results = []
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    results.append(env.quick_results)

stacked_results = np.stack(results)

results.append(np.array([str(env.EVs_n)+"_EVs_mean", np.mean(stacked_results[:, 1].astype('float32')), np.mean(stacked_results[:, 2].astype('float32')), np.mean(stacked_results[:, 3].astype('float32')), np.mean(stacked_results[:, 4].astype('float32')), np.mean(stacked_results[:, 5].astype('float32'))]))
results_table.extend(results)
#Save the VPP table
#VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')
#VPP_table = env.VPP_table

- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  26402.15 , over-consume=kWh  45924.4 , under-consume=kWh  19522.25 , Total_cost=€  1174.54 , overcost=€  1976.71 , Av.EV_en_left=kWh  100.0 , Charging_events=  1825 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  67.66
Simulating VPP....
- VPP.Simulation results
 LOAD_INFO: Sum_Energy=KWh  -2160.87 , over-consume=KWh  696.31 , under-consume=KWh  2857.18 , Total_cost=€  -84.83 , Overcost=€  30.07 
 EV_INFO: Av.EV_energy_leaving=kWh  65.35 , Std.EV_energy_leaving=kWh  23.33 , EV_departures =  1824 , EV_queue_left =  0
SCORE:  Cumulative_reward= 519531.42 - Step_rewars (load_t= 448859.63, EVs_energy_t= 22605.22)
 - Final_rewards (EVs_energy= 24778.27, Overconsume= 4837.91, Underconsume= 1341.12, Overcost= 17109.27)
Episode:1 Score:519531.4164197713
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  25945.91 , over-consume=kWh  45031.4 , under-consume=kWh  19085.49 , T

## Validating dataset VPP Simulation using the loaded trained model [30 EVs per week]

In [10]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_validating_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
results = []
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    results.append(env.quick_results)

stacked_results = np.stack(results)
results.append(np.array([str(env.EVs_n)+"_EVs_mean", np.mean(stacked_results[:, 1].astype('float32')), np.mean(stacked_results[:, 2].astype('float32')), np.mean(stacked_results[:, 3].astype('float32')), np.mean(stacked_results[:, 4].astype('float32')), np.mean(stacked_results[:, 5].astype('float32'))]))
results_table.extend(results)
#Save the VPP table
#VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')
#VPP_table = env.VPP_table

Charging event: 21901, Arrival time: 2022-01-01 11:00:00, Parking_time: 24, Leaving_time: 2022-01-02 11:00:00, SOC: 0.5358505668202658, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 23465, Arrival time: 2022-12-31 15:45:00, Parking_time: 22.757028553519497, Leaving_time: 2023-01-01 14:30:25.302793, SOC: 0.47193792480724495, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -30085.39 , over-consume=kWh  2136.67 , under-consume=kWh  -32222.06 , Total_cost=€  -1187.15 , overcost=€  113.34
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  22087.01 , over-consume=kWh  42215.6 , under-consume=kWh  20128.59 , Total_cost=€  1013.98 , overcost=€  1838.76 , Charging_events=  1565 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  70.59
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  21744.39 , over-consume=kWh  41768.29 , under-consume=kWh  20023.91 , Total_cost=€  

## Validating dataset VPP Simulation using the loaded trained model [25 EVs per week]

In [11]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_validating_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
results = []
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    results.append(env.quick_results)

stacked_results = np.stack(results)
results.append(np.array([str(env.EVs_n)+"_EVs_mean", np.mean(stacked_results[:, 1].astype('float32')), np.mean(stacked_results[:, 2].astype('float32')), np.mean(stacked_results[:, 3].astype('float32')), np.mean(stacked_results[:, 4].astype('float32')), np.mean(stacked_results[:, 5].astype('float32'))]))
results_table.extend(results)
#Save the VPP table
#VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')
#VPP_table = env.VPP_table

Charging event: 40681, Arrival time: 2022-01-01 09:15:00, Parking_time: 24, Leaving_time: 2022-01-02 09:15:00, SOC: 0.3906378187053575, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 41984, Arrival time: 2022-12-31 16:00:00, Parking_time: 24, Leaving_time: 2023-01-01 16:00:00, SOC: 0.6959206456700326, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -30085.39 , over-consume=kWh  2136.67 , under-consume=kWh  -32222.06 , Total_cost=€  -1187.15 , overcost=€  113.34
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  18187.46 , over-consume=kWh  39127.3 , under-consume=kWh  20939.84 , Total_cost=€  844.66 , overcost=€  1705.98 , Charging_events=  1304 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  74.71
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  19553.59 , over-consume=kWh  39927.0 , under-consume=kWh  20373.41 , Total_cost=€  920.12 , overcost=€  1747.

## Validating dataset VPP Simulation using the loaded trained model [20 EVs per week]

In [12]:
case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_validating_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
results = []
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    results.append(env.quick_results)

stacked_results = np.stack(results)
results.append(np.array([str(env.EVs_n)+"_EVs_mean", np.mean(stacked_results[:, 1].astype('float32')), np.mean(stacked_results[:, 2].astype('float32')), np.mean(stacked_results[:, 3].astype('float32')), np.mean(stacked_results[:, 4].astype('float32')), np.mean(stacked_results[:, 5].astype('float32'))]))
results_table.extend(results)
#Save the VPP table
#VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')
#VPP_table = env.VPP_table

Charging event: 56329, Arrival time: 2022-01-01 05:45:00, Parking_time: 23.477641880175405, Leaving_time: 2022-01-02 05:13:39.510769, SOC: 0.5804631404697417, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 57371, Arrival time: 2022-12-30 10:30:00, Parking_time: 24, Leaving_time: 2022-12-31 10:30:00, SOC: 0.5809175302665704, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -30085.39 , over-consume=kWh  2136.67 , under-consume=kWh  -32222.06 , Total_cost=€  -1187.15 , overcost=€  113.34
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  12849.21 , over-consume=kWh  34354.4 , under-consume=kWh  21505.19 , Total_cost=€  597.47 , overcost=€  1483.86 , Charging_events=  1043 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  80.89
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  12322.17 , over-consume=kWh  34174.31 , under-consume=kWh  21852.14 , Total_cost=€  60

## Validating dataset VPP Simulation using the loaded trained model [15 EVs per week]


In [13]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_validating_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
results = []
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    results.append(env.quick_results)

stacked_results = np.stack(results)
results.append(np.array([str(env.EVs_n)+"_EVs_mean", np.mean(stacked_results[:, 1].astype('float32')), np.mean(stacked_results[:, 2].astype('float32')), np.mean(stacked_results[:, 3].astype('float32')), np.mean(stacked_results[:, 4].astype('float32')), np.mean(stacked_results[:, 5].astype('float32'))]))
results_table.extend(results)
#Save the VPP table
#VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')
#VPP_table = env.VPP_table

Charging event: 68845, Arrival time: 2022-01-01 10:30:00, Parking_time: 24, Leaving_time: 2022-01-02 10:30:00, SOC: 0.49069367192535096, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 69627, Arrival time: 2022-12-31 21:30:00, Parking_time: 24, Leaving_time: 2023-01-01 21:30:00, SOC: 0.5740481074786997, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -30085.39 , over-consume=kWh  2136.67 , under-consume=kWh  -32222.06 , Total_cost=€  -1187.15 , overcost=€  113.34
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  5232.36 , over-consume=kWh  28927.82 , under-consume=kWh  23695.46 , Total_cost=€  292.97 , overcost=€  1255.18 , Charging_events=  783 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  91.15
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  5121.57 , over-consume=kWh  28639.33 , under-consume=kWh  23517.76 , Total_cost=€  297.23 , overcost=€  1256.

## Validating dataset VPP Simulation using the loaded trained model [10 EVs per week]


In [14]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_validating_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
results = []
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    results.append(env.quick_results)

stacked_results = np.stack(results)
results.append(np.array([str(env.EVs_n)+"_EVs_mean", np.mean(stacked_results[:, 1].astype('float32')), np.mean(stacked_results[:, 2].astype('float32')), np.mean(stacked_results[:, 3].astype('float32')), np.mean(stacked_results[:, 4].astype('float32')), np.mean(stacked_results[:, 5].astype('float32'))]))
results_table.extend(results)
#Save the VPP table
#VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')
#VPP_table = env.VPP_table

Charging event: 78241, Arrival time: 2022-01-01 14:30:00, Parking_time: 24, Leaving_time: 2022-01-02 14:30:00, SOC: 0.35677934835250813, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 78762, Arrival time: 2022-12-31 08:00:00, Parking_time: 23.51481026352881, Leaving_time: 2023-01-01 07:30:53.316949, SOC: 0.35355365801832206, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -30085.39 , over-consume=kWh  2136.67 , under-consume=kWh  -32222.06 , Total_cost=€  -1187.15 , overcost=€  113.34
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  -4715.61 , over-consume=kWh  21227.86 , under-consume=kWh  25943.48 , Total_cost=€  -112.35 , overcost=€  944.49 , Charging_events=  522 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  111.73
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  -5079.1 , over-consume=kWh  20548.26 , under-consume=kWh  25627.36 , Total_cost=€  -

In [27]:
dataframe_results = pd.DataFrame(np.stack(results_table))
dataframe_results = dataframe_results.rename(columns={0:"Name",1:"underconsume",2:"overconsume",3:"overcost",4:"av_EV_energy_left",5:"cumulative_reward"})

dataframe_results.to_csv("data/algorithms_results/algorithms_results_table/EV_experiments_validating.csv")

dataframe_results.head()

,Name,underconsume,overconsume,overcost,av_EV_energy_left,cumulative_reward
0,35_EVs,2857.181880443665,696.3097377826446,30.074379495280297,65.35106042268241,519531.4164197719
1,35_EVs,3340.6915204211887,583.3803268578152,24.751925045901096,64.89275315700466,515537.6004607147
2,35_EVs,2998.2668386812306,665.3212970453882,26.908554428165836,64.90959564618335,518365.2336867316
3,35_EVs,2761.939994345115,606.3109830296969,26.35926840426527,65.91218258184632,530654.3600571246
4,35_EVs,3102.0401707939845,550.0186949057703,24.392065040873916,65.19775177169113,521500.5385854199
